In [68]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
size = 200000

In [69]:
train_url = '/kaggle/input/revised-corrector-dataset/train_corr.csv'
test_url = '/kaggle/input/revised-corrector-dataset/test_corr.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/kaggle/input/bangla-stopwords/stopwords_bangla.xlsx',index_col=False)

In [70]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [71]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [72]:
from sklearn.model_selection import train_test_split
MAX_FEATURES = 6000
EMBED_SIZE = 128
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(df_train['Comment'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['Comment'])
list_tokenized_test = tokenizer.texts_to_sequences(df_test['Comment'])
RNN_CELL_SIZE = 32
MAX_LEN = 60   # Since our mean length is 56.6
X_train = pad_sequences(list_tokenized_train, maxlen=MAX_LEN)
X_test = pad_sequences(list_tokenized_test,maxlen=MAX_LEN)
y_train = df_train['Error']
y_test = df_test['Error']

In [73]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
 
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [74]:
sequence_input = Input(shape=(MAX_LEN,), dtype="int32")
embedded_sequences = Embedding(MAX_FEATURES, EMBED_SIZE)(sequence_input)

In [75]:
lstm = Bidirectional(LSTM(RNN_CELL_SIZE, return_sequences = True), name="bi_lstm_0")(embedded_sequences)# Getting our LSTM outputs
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(RNN_CELL_SIZE, return_sequences=True, return_state=True), name="bi_lstm_1")(lstm)

In [76]:
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
context_vector, attention_weights = Attention(10)(lstm, state_h)
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.05)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = keras.Model(inputs=sequence_input, outputs=output)

In [77]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)

In [78]:
BATCH_SIZE = 100
EPOCHS = 5
history = model.fit(X_train,y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_split=0.2)

Epoch 1/5
161/161 [==============================] - 34s 150ms/step - loss: 0.6701 - tp: 7090.0000 - fp: 5577.0000 - tn: 2062.0000 - fn: 1337.0000 - accuracy: 0.5697 - precision: 0.5597 - recall: 0.8413 - auc: 0.5984 - val_loss: 0.9126 - val_tp: 1798.0000 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 2219.0000 - val_accuracy: 0.4476 - val_precision: 1.0000 - val_recall: 0.4476 - val_auc: 0.0000e+00
Epoch 2/5
161/161 [==============================] - 9s 58ms/step - loss: 0.5072 - tp: 6212.0000 - fp: 1723.0000 - tn: 5916.0000 - fn: 2215.0000 - accuracy: 0.7549 - precision: 0.7829 - recall: 0.7372 - auc: 0.8287 - val_loss: 0.7781 - val_tp: 2121.0000 - val_fp: 0.0000e+00 - val_tn: 0.0000e+00 - val_fn: 1896.0000 - val_accuracy: 0.5280 - val_precision: 1.0000 - val_recall: 0.5280 - val_auc: 0.0000e+00
Epoch 3/5
161/161 [==============================] - 7s 41ms/step - loss: 0.4125 - tp: 6669.0000 - fp: 1174.0000 - tn: 6465.0000 - fn: 1758.0000 - accuracy: 0.8175 - precision: 0.8503 - 

In [79]:
pred = (model.predict(X_test) > 0.5).astype("int32")

157/157 [==============================] - 2s 6ms/step


In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred, target_names = ['Correct','Incorrect']))

              precision    recall  f1-score   support

     Correct       0.52      0.65      0.58      1910
   Incorrect       0.75      0.63      0.68      3112

    accuracy                           0.64      5022
   macro avg       0.63      0.64      0.63      5022
weighted avg       0.66      0.64      0.64      5022

